<a href="https://colab.research.google.com/github/Muhaiminul8473/Bengali-automated-notes-generation-service/blob/main/Bengali_automated_notes_writing_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 55kB/s 
     |████████████████████████████████| 3.2MB 50.7MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
     |████████████████████████████████| 368kB 38.8MB/s 
  Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.0
    Uninstalling tensorflow-2.4.0:
      Successfully uninstalled tensorflow-2.4.0


In [3]:
pip install tflearn

     |████████████████████████████████| 112kB 11.5MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-cp36-none-any.whl size=127301 sha256=59b4c90f8612a6a94ef5726e4655f48ed470410943441e1dcab1dfae874aea4a
  Stored in directory: /root/.cache/pip/wheels/31/d2/ed/fb9a0d301dd9586c11e9547120278e624227f22fd5f4baf744
Successfully built tflearn


In [5]:
#Used in Tensorflow Model
import numpy as np
import tensorflow as tf
import tflearn
import random

#Usde to for Contextualisation and Other NLP Tasks.
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Other
import json
import pickle
import warnings
warnings.filterwarnings("ignore")

In [6]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
print(tf.__version__)


1.13.2


In [8]:
print("Processing the Intents.....")
with open('/content/gdrive/My Drive/Final defence/intents2.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [9]:
words = []
classes = []
documents = []
ignore_words = ['?']
print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [10]:
print(words)

['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত', 'শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'বিএনপি', 'অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'সমাবেশ', 'করবে', 'বিএনপি', 'ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'আগুন', 'প্লাস্টিক', 'কারখানায়', 'আগুন', 'ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'খুন', 'আবাসিক', 'হোটেলে', 'খুন', 'ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ', 'শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম', 'নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম', 'জামালপুরে', 'বিদ্যুৎস্পৃষ্ট', 'হয়ে', 'কলেজছাত্রীর', 'মৃত্যু', 'কলেজছাত্রীর', 'মৃত্যু', 'জামালপুরে', 'বিদ্যুৎস্পৃষ্ট', 'হয়ে', 'ঢাকা-চট্টগ্রাম', 'ও', 'ঢাকা-ময়মনসিংহ', 'মহাসড়কের', '

In [11]:
print(documents)

[(['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত'], 'নিহত'), (['চত্বরে', 'বাসের', 'ধাক্কায়', 'নিহত'], 'নিহত'), (['শাপলা', 'চত্বরে', 'বাসের', 'ধাক্কায়'], 'নিহত'), (['অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে', 'বিএনপি'], 'বিএনপি'), (['অনুমতি', 'না', 'পেলেও', 'সমাবেশ', 'করবে'], 'বিএনপি'), (['সমাবেশ', 'করবে', 'বিএনপি'], 'বিএনপি'), (['ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়', 'আগুন'], 'আগুন'), (['প্লাস্টিক', 'কারখানায়', 'আগুন'], 'আগুন'), (['ইসলামবাগে', 'প্লাস্টিক', 'কারখানায়'], 'আগুন'), (['ফকিরাপুলে', 'আবাসিক', 'হোটেলে', 'খুন'], 'খুন'), (['আবাসিক', 'হোটেলে', 'খুন'], 'খুন'), (['ফকিরাপুলে', 'আবাসিক', 'হোটেলে'], 'খুন'), (['শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ'], 'বিলবোর্ড'), (['অবৈধ', 'বিলবোর্ড', 'সরাতে', 'পুলিশের', 'নির্দেশ'], 'বিলবোর্ড'), (['শনিবারের', 'মধ্যে', 'অবৈধ', 'বিলবোর্ড'], 'বিলবোর্ড'), (['নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়', ':', 'নাসিম'], 'নির্বাচন'), (['নির্বাচন', 'কমিশন', 'ছাড়া', 'কারও', 'সঙ্গে', 'সংলাপ', 'নয়'], 'নির্বাচন'), (['

In [12]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
27 documents
9 classes ['আগুন', 'খুন', 'নবজাতক', 'নির্বাচন', 'নিহত', 'বিএনপি', 'বিদ্যুৎস্পৃষ্ট', 'বিলবোর্ড', 'মহাসড়কের']
55 unique stemmed words [':', 'অনুমতি', 'অবৈধ', 'আগুন', 'আবাসিক', 'ইসলামবাগে', 'ও', 'কমিশন', 'করবে', 'কলেজছাত্রীর', 'কাজ', 'কারও', 'কারখানায়', 'কোলে', 'খুন', 'গেল', 'চত্বরে', 'চুরি', 'ছাড়া', 'জামালপুরে', 'জুনে', 'ঢাকা-চট্টগ্রাম', 'ঢাকা-ময়মনসিংহ', 'ধাক্কায়', 'নবজাতক', 'নয়', 'না', 'নাসিম', 'নির্দেশ', 'নির্বাচন', 'নিহত', 'পুলিশের', 'পেলেও', 'প্লাস্টিক', 'ফকিরাপুলে', 'ফিরে', 'বাসের', 'বিএনপি', 'বিদ্যুৎস্পৃষ্ট', 'বিলবোর্ড', 'মধ্যে', 'মহাসড়কের', 'মায়ের', 'মৃত্যু', 'শনিবারের', 'শাপলা', 'শেষ', 'সংলাপ', 'সঙ্গে', 'সমাবেশ', 'সরাতে', 'হওয়া', 'হবে', 'হয়ে', 'হোটেলে']


In [13]:
print(len(words))

55


In [14]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Traning Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Traning Set, Bag of Words for our Model....


In [15]:
print(len(words))

55


In [16]:
print(len(pattern_words))

3


In [17]:
print(len(bag))

55


In [18]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")
tf.reset_default_graph()

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [19]:
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)
print("Training....")

Training....


In [20]:
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Use tf.cast instead.


In [21]:
print("Training the Model.......")
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
print("Saving the Model.......")
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00290 | time: 0.028s
| Adam | epoch: 1000 | loss: 0.00290 - acc: 1.0000 -- iter: 24/27
Training Step: 4000  | total loss: 0.00294 | time: 0.032s
| Adam | epoch: 1000 | loss: 0.00294 - acc: 1.0000 -- iter: 27/27
--
Saving the Model.......
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [23]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']


with open('/content/gdrive/My Drive/Final defence/intents2.json') as json_data:
    intents = json.load(json_data)
    
print("Loading the Model......")
# load our saved model
model.load('./model.tflearn')

Loading Pickle.....
Loading the Model......
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [24]:
def clean_up_sentence(sentence):
    # It Tokenize or Break it into the constituents parts of Sentense.
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in intents['intents']:
                # Tag Finding
                if i['tag'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print(random.choice(i['responses']))

            results.pop(0)

ERROR_THRESHOLD = 0.25


In [ ]:
while True:
    input_data = input("You- ")
    answer = response(input_data)
    answer

You- বাসের ধাক্কায় নিহত
রাজধানীর মতিঝিলের শাপলা চত্বর এলাকায় আজ বৃহস্পতিবার সন্ধ্যায় যাত্রীবাহী বাসের ধাক্কায় রঞ্জন দেবনাথ (৩৯) নামের এক ব্যক্তি নিহত হয়েছেন।নিহত রঞ্জন দেবনাথের স্ত্রী বীণা বিশ্বাস ঢাকা মেডিকেল কলেজ হাসপাতালের স্টাফ নার্স। তিনি জানান, রঞ্জন দেবনাথ আফতাব গ্রুপে চাকরি করেন। 
You- শনিবারের মধ্যে অবৈধ বিলবোর্ড সরাতে
চট্টগ্রাম নগরে স্থাপিত অবৈধ সব বিলবোর্ড আগামী শনিবারের মধ্যে অপসারণ করতে বলা হয়েছে। অন্যথায় পরের দিন থেকে বিলবোর্ড স্থাপনকারী মালিকদের গ্রেপ্তার করে আইনি ব্যবস্থা নেওয়া হবে বলে মহানগর পুলিশের পাঠানো এক বিজ্ঞপ্তিতে জানানো হয়েছে।
